# Download the images from the tweets and push to s3 for scalable CDN
(Reference: https://github.com/INFO-4602-5602/final-project-tweetsonamap/issues/2)

In [1]:
import pymongo, json, pprint, urllib.request, os.path, sys
from html.parser import HTMLParser

Connect to the DB

    client = pymongo.MongoClient('mongodb://epic-analytics.cs.colorado.edu')
    db = client.matthew
    print(db.collection_names())
    tweets = db.tweets
    print(tweets.count(), "tweets")

In [2]:
client = pymongo.MongoClient('localhost')
tweets = client['matthew'].tweets

###  Image Handlers

In [47]:
err_log = open('scraping_errors_twitter.log','w')

class InstagramHandler(HTMLParser):
    """
        Custom function to handle scraping images from Instagram... totally violates TOS, I think.
    """
    def __init__(self, tweet_id):
        self.image_url = None
        self.tweet_id = tweet_id
        self.directory = "/data/infovis/instagram"
        super().__init__()
        
    def image_needed(self):
        if os.path.exists(self.directory+"/"+self.tweet_id+".jpg"):
            err_log.write("File Exists: "+self.tweet_id+"\n")
            return False
        else:
            return True

    def handle_starttag(self, tag, attrs):
        if tag=='meta':
            if attrs[0][1]=='og:image':
                self.image_url = attrs[1][1]
                try:
                    req = urllib.request.urlopen(self.image_url)
                    with open(self.directory+"/"+self.tweet_id+".jpg",'wb') as pic:
                        pic.write(req.read())
                except:
                    err_log.write("Error, image not found: "+self.tweet_id+"\n")
                    
class TwitterHandler():
    """
        Custom function to handle scraping images from Twitter...
    """
    def __init__(self, tweet_id):
        self.image_url = None
        self.tweet_id = tweet_id
        self.directory = "/data/infovis/twitter"
        
    def image_needed(self):
        if os.path.exists(self.directory+"/"+self.tweet_id+".jpg"):
            err_log.write("File Exists: "+self.tweet_id+"\n")
            return False
        else:
            return True

    def download(self, url):
        print(url)
        return url
        try:
            req = urllib.request.urlopen(url)
            with open(self.directory+"/"+self.tweet_id+".jpg",'wb') as pic:
                pic.write(req.read())
        except:
            err_log.write("Error, image not found: "+self.tweet_id+"\n")

In [48]:
cursor = tweets.find({'geo' : {"$exists":True}})

In [49]:
geo_image_tweets = {
    "verb":"post", # original tweet
    "twitter_extended_entities.media.0":{"$exists":True}, # at least 1 media entity
    "twitter_extended_entities.media": {"$all":[{"$elemMatch": { "type": "photo" }}]}, # all media entities are photos
    "location.geo.coordinates":{"$exists":True} # has geolocation (not necessarily a point)
}
cursor = tweets.find(geo_image_tweets).limit(10)

In [50]:
idx = 0;
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    for m, entities in t['twitter_extended_entities'].items():
        for idx, entity in enumerate(entities):
            downloader = TwitterHandler(tweet_id)
            if downloader.image_needed():
                downloader.download(entity['media_url'])
                    
        #Instagram Case
#         if url['expanded_url'].startswith("https://www.instagram.com"):            
#             parser = InstagramHandler(tweet_id)
#             if parser.image_needed():
#                 try:
#                     req = urllib.request.urlopen(url['expanded_url'])
#                     parser.feed(req.read().decode('utf-8'))
#                 except:
#                     err_log.write("Error, page not found: "+tweet_id + "\n")

    #Twitter Case
err_log.close()

http://pbs.twimg.com/media/CtKGGh_XgAAZiZB.jpg
http://pbs.twimg.com/media/CtKGvDIUsAAO_ku.jpg
http://pbs.twimg.com/media/CtKHV4yWIAABbJY.jpg
http://pbs.twimg.com/media/CtKKt44VYAAIs5w.jpg
http://pbs.twimg.com/media/CtKPWsrVMAAz1za.jpg
http://pbs.twimg.com/media/B2hfaeSIMAEJq5d.jpg
http://pbs.twimg.com/media/BuR8ET2IcAEY1aR.jpg
http://pbs.twimg.com/media/CtKUcX-VMAA9ENY.jpg
http://pbs.twimg.com/media/CtKWEuiXEAArmSP.jpg
http://pbs.twimg.com/media/CtKW9zhUAAAuzfa.jpg
